In [1]:
import os
import json
from dataclasses import dataclass
from typing import Dict, List, Any, Tuple

import numpy as np
import torch
from torch.utils.data import Dataset

import av  # pip install av
from transformers import (
    AutoImageProcessor,
    VideoMAEForVideoClassification,
    TrainingArguments,
    Trainer,
)

import evaluate  # pip install evaluate

# load environment variables with dotenv
from dotenv import load_dotenv
load_dotenv()


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
from train import *
import os


In [3]:
# Point this at the Olympic Boxing dataset directory
DATASET_DIR = "Olympic Boxing Punch Classification Video Dataset"

# Pretrained VideoMAE base (self-supervised on K400)
model_name = "MCG-NJU/videomae-base"

image_processor = AutoImageProcessor.from_pretrained(model_name)
model = VideoMAEForVideoClassification.from_pretrained(
    model_name,
    num_labels=len(LABEL2ID),
    label2id=LABEL2ID,
    id2label=ID2LABEL,
)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# check for cuda
if torch.cuda.is_available():
    model.to("cuda")
    print("Using CUDA")
else:
    print("Using CPU")

Using CUDA


In [5]:
train_dataset = BoxingDataset(
    split="train",

)
val_dataset = BoxingDataset(
    split="val",
)
test_dataset = BoxingDataset(
    split="test",
)

In [6]:

# FACTS used batch_size=4, grad_accum=2, warmup_ratio=0.1, epochs=10
# Learning rate is not rendered in the HTML; start with 1e-4 and tune around it.
training_args = TrainingArguments(
    output_dir="./facts-boxing-videomae",
    eval_strategy="steps",
    eval_steps=500,
    save_steps=500,
    logging_steps=100,
    save_total_limit=2,
    num_train_epochs=10,
    per_device_train_batch_size=4, 
    per_device_eval_batch_size=8, 
    gradient_accumulation_steps=2,  # effective batch size 8
    warmup_ratio=0.1,
    learning_rate=1e-4,
    weight_decay=0.05,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    report_to="wandb",  # or "wandb"/"tensorboard"
    dataloader_num_workers=4,        # ADD THIS - use multiple workers
    dataloader_pin_memory=True,      # ADD THIS - faster CPU->GPU transfer
    dataloader_prefetch_factor=2, 
)

data_collator = VideoDataCollator()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [7]:

# Train
trainer.train()


wandb: Currently logged in as: nkosik11 (nkosik11-hobby) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy,Macro F1,F1 Lhhp,Precision Lhhp,Recall Lhhp,F1 Rhhp,Precision Rhhp,Recall Rhhp,F1 Lhmp,Precision Lhmp,Recall Lhmp,F1 Rhmp,Precision Rhmp,Recall Rhmp,F1 Lhblp,Precision Lhblp,Recall Lhblp,F1 Rhblp,Precision Rhblp,Recall Rhblp,F1 Lhbp,Precision Lhbp,Recall Lhbp,F1 Rhbp,Precision Rhbp,Recall Rhbp
500,1.805200,1.806617,0.367615,0.067200,0.537600,0.367615,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1000,1.807400,1.775429,0.367615,0.067200,0.537600,0.367615,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1500,1.803900,1.782106,0.367615,0.067200,0.537600,0.367615,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2000,1.816800,1.903053,0.323851,0.087020,0.480591,0.348525,0.773810,0.215569,0.214286,0.216867,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2500,1.747300,1.799748,0.367615,0.067200,0.537600,0.367615,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3000,1.689200,2.244027,0.085339,0.031923,0.194444,0.233333,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060942,0.033846,0.305556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3500,1.661300,2.548012,0.177243,0.047164,0.009569,0.024390,0.005952,0.338330,0.205729,0.951807,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.029412,0.031250,0.027778,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4000,1.599800,2.512360,0.223195,0.081851,0.411168,0.358407,0.482143,0.130435,0.163636,0.108434,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.113208,0.068182,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4500,1.392500,2.754348,0.185996,0.082335,0.308176,0.326667,0.291667,0.246154,0.214286,0.289157,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.104348,0.061856,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


TrainOutput(global_step=4610, training_loss=1.7079626623306772, metrics={'train_runtime': 538.6377, 'train_samples_per_second': 68.376, 'train_steps_per_second': 8.559, 'total_flos': 4.589501448796766e+19, 'train_loss': 1.7079626623306772, 'epoch': 10.0})

In [9]:

# Evaluate on test split
test_metrics = trainer.evaluate(test_dataset)
print("Test metrics:", test_metrics)


Test metrics: {'eval_loss': 1.8251409530639648, 'eval_accuracy': 0.3618843683083512, 'eval_macro_f1': 0.06643081761006289, 'eval_f1_LHHP': 0.5314465408805031, 'eval_precision_LHHP': 0.3618843683083512, 'eval_recall_LHHP': 1.0, 'eval_f1_RHHP': 0.0, 'eval_precision_RHHP': 0.0, 'eval_recall_RHHP': 0.0, 'eval_f1_LHMP': 0.0, 'eval_precision_LHMP': 0.0, 'eval_recall_LHMP': 0.0, 'eval_f1_RHMP': 0.0, 'eval_precision_RHMP': 0.0, 'eval_recall_RHMP': 0.0, 'eval_f1_LHBlP': 0.0, 'eval_precision_LHBlP': 0.0, 'eval_recall_LHBlP': 0.0, 'eval_f1_RHBlP': 0.0, 'eval_precision_RHBlP': 0.0, 'eval_recall_RHBlP': 0.0, 'eval_f1_LHBP': 0.0, 'eval_precision_LHBP': 0.0, 'eval_recall_LHBP': 0.0, 'eval_f1_RHBP': 0.0, 'eval_precision_RHBP': 0.0, 'eval_recall_RHBP': 0.0, 'eval_runtime': 3.6572, 'eval_samples_per_second': 127.694, 'eval_steps_per_second': 16.133, 'epoch': 10.0}
